# RSA: LeNet-5 x FashionMNIST

*Version History*

* **5.0** - Check functionality of ``RepresentationalSimilarityTrainer`` and RSA functions via test run


## Preparation


### Imports & Constants

In [1]:
import random
from datetime import datetime
from functools import partial
from pathlib import Path
from typing import Any, Callable, Dict, Literal, Optional, Tuple

import numpy as np
import torch
from omegaconf import OmegaConf
from torch import linalg as LA
from torch import nn
from torch.nn import functional as F
from torchmetrics import MetricCollection
from torchmetrics.classification import MulticlassAccuracy

from src.base_classes.base_loader import BaseLoader
from src.datasets.fashionmnist import FashionMNIST
from src.models.lenet import LeNet
from src.training.representational_similarity_trainer import RepresentationalSimilarityTrainer
from src.utils.classification_presets import ClassificationPresets
from src.utils.utils import evaluate_classifier


# Paths
DATA_DIR = "../data"
CHKPT_PATH = "../models/lenet_fashionmnist.pt"

# Seeds
TORCH_SEED = 89
SHUFFLE_SEED = 858
SPLIT_SEED = 42

# Dataloader
VAL_SPLIT = 0.2
BATCH_SIZE = 128
NUM_WORKERS = 0

# Training
DEVICE = torch.device(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

### Helper Functions

* ``get_timestamp`` &rarr; Get a timestamp of the current date and time.
* ``set_seeds`` &rarr; Set random seeds for reproducibility.

In [2]:
def get_timestamp() -> str:
    """Get a timestamp of the current date and time."""
    
    return datetime.now().strftime("%Y-%m_%d-%H-%M-%S")


def set_seeds() -> None:
    """Set random seeds for reproducibility."""
    
    random.seed(TORCH_SEED)
    np.random.seed(TORCH_SEED)
    torch.manual_seed(TORCH_SEED)
    torch.cuda.manual_seed_all(TORCH_SEED)


### FashionMNIST Dataset

In [3]:
# Prepare transforms
train_transform = ClassificationPresets(
    mean=[0.2860],
    std=[0.3205],
    crop_size=32,
    crop_scale=(0.8, 0.8),
    crop_ratio=(1, 1),
    flip_prob=0.5
)

val_transform = ClassificationPresets(
    mean=[0.2860],
    std=[0.3205],
    crop_size=32,
    resize_size=32,
    is_training=False
)

# Load FashionMNIST dataset
fashionmnist = FashionMNIST(
    data_dir=DATA_DIR,
    train=True,
    load_into_memory=True
)

# Prepare dataloaders
base_loader = BaseLoader(
    dataset=fashionmnist,
    main_transform=train_transform,
    val_transform=val_transform,
    val_split=VAL_SPLIT,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    split_seed=SPLIT_SEED,
    shuffle_seed=SHUFFLE_SEED
)
TRAIN_LOADER = base_loader.get_dataloader(mode="Main")
VAL_LOADER = base_loader.get_dataloader(mode="Val")

### LeNet-5

* ``load_pretrained_lenet`` &rarr; Load the pre-trained LeNet-5 network.

In [4]:
def load_pretrained_lenet(
    is_training: bool = False
) -> nn.Module:
    """Load the pre-trained LeNet-5 network.

    Args:
        is_training: If True, the network is set to training mode and
          gradients are enabled.  If False, the network is set to
          evaluation mode and gradients are disabled.

    Returns:
        The pre-trained LeNet-5 network.
    """

    # Initialize network
    lenet = LeNet().to(DEVICE)

    # Load pre-trained weights
    lenet.load_state_dict(
        torch.load(
            CHKPT_PATH,
            map_location=DEVICE,
            weights_only=False
        )["model_state_dict"]
    )

    # Set to evaluation mode and disable gradients, if applicable
    if not is_training:
        lenet.eval()
        lenet.requires_grad_(False)
    
    return lenet


Next, we initialize the LeNet-5 network with pre-trained weights, and check its performance.

In [5]:
# Criterion used to evaluate classification performance of LeNet-5
criterion = nn.CrossEntropyLoss()

# Additional metrics used to evaluate LeNet-5
PREDICTION_METRICS = MetricCollection({
    "Accuracy@1": MulticlassAccuracy(
        num_classes=10,
        top_k=1,
        average="micro",
        multidim_average="global"
    ),
    "Accuracy@5": MulticlassAccuracy(
        num_classes=10,
        top_k=5,
        average="micro",
        multidim_average="global"
    )
})

# Evaluate performance
results = evaluate_classifier(
    model=load_pretrained_lenet(),
    test_loader=VAL_LOADER,  # same validation set used during training
    criterion=criterion,
    metrics=PREDICTION_METRICS,
    device=DEVICE
)

# Load meta-data from checkpoint
epoch_idx = torch.load(
    CHKPT_PATH,
    map_location=DEVICE,
    weights_only=False
)["epoch_idx"]
output_text = [f"Results on validation set after {epoch_idx} epochs:"]

# Print results
for metric, value in results.items():
    output_text.append(f"{metric}: {value:.3f}")
print("\n  ".join(output_text))

Evaluating LeNet: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 94/94 [00:01<00:00, 75.38batch/s]

Results on validation set after 147 epochs:
  Loss: 0.256
  Accuracy@1: 0.906
  Accuracy@5: 0.999


In [6]:
print(load_pretrained_lenet())

LeNet(
  (net): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): ReLU()
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): ReLU()
    (8): Rearrange('b c h w -> b (c h w)', c=120, h=1, w=1)
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): ReLU()
    (11): Linear(in_features=84, out_features=10, bias=True)
  )
)


### Training Configuration

Next, we specify those parameters of the training configuration ``cfg`` used by the ``RepresentationalSimilarityTrainer``  
class that shouldn't change across experiments. **Parameters that have to be set individually for each experiment are**:

* hooks.ref
* hooks.train
* optimizer.params
* paths.checkpoints
* paths.tensorboard
* performance.patience
* training.num_epochs

**When training is to be resumed from a saved checkpoint**, the following parameter needs to be set as well:

* training.resume_from

In [7]:
cfg_dict = {
    "checkpoints": {
        "delete_previous": True,
        "save_best_model": True,
        "save_frequency": 5
    },
    "dataloader": {
        "batch_size": BATCH_SIZE
    },
    "model": {
        "name": "lenet"
    },
    "optimizer": {
        "name": "SGD"
    },
    "performance": {
        "dataset": "Val",
        "higher_is_better": False,
        "keep_previous_best_score": True,
        "metric": "Loss"
    },
    "tensorboard": {
        "updates_per_epoch": {
            "Train": 10,
            "Val": 10
        }
    },
    "training": {
        "resume_from": None
    }
}

# Convert to DictConfig and inspect configuration
cfg = OmegaConf.create(cfg_dict)
print(f"\033[1mGeneral configuration\033[0m:\n\n{OmegaConf.to_yaml(cfg)}")

General configuration:

checkpoints:
  delete_previous: true
  save_best_model: true
  save_frequency: 5
dataloader:
  batch_size: 128
model:
  name: lenet
optimizer:
  name: SGD
performance:
  dataset: Val
  higher_is_better: false
  keep_previous_best_score: true
  metric: Loss
tensorboard:
  updates_per_epoch:
    Train: 10
    Val: 10
training:
  resume_from: null



## RSA


### Helper Functions

* ``_get_upper_tri_matrix`` &rarr; Extract the upper triangular part of a square matrix.
* ``_validate_activations`` &rarr; Validate activations from which to compute an RDM.
* ``_is_vector`` &rarr; Check if the input is a ``torch.Tensor`` of dimension 1.

In [8]:
def _get_upper_tri_matrix(A: torch.Tensor) -> torch.Tensor:
    """Extract the upper triangular part of a square matrix.

    Args:
        A: The square matrix from which to extract the upper triangular
          matrix (excluding the diagonal).

    Returns:
        The upper triangular matrix (excluding the diagonal) of the
        square matrix ``A``, flattened into a vector using row-major
        order.
    """

    mask = torch.triu(torch.ones_like(A, dtype=torch.bool), diagonal=1)
    return A[mask]


def _validate_activations(activations: torch.Tensor) -> None:
    """Validate activations from which to compute an RDM.

    This function is meant to be called on a matrix of activations from
    which to compute an RDM.  Rows of the matrix are associated with
    different stimuli, while columns correspond to unit activations.
    N stimuli give rise to N * (N - 1) / 2 distinct pairwise distances.
    Hence, there should be activations for at least N >= 3 stimuli.

    Args:
        activations: The matrix of activations from which to compute an
          RDM.  Must be a 2-D tensor of size (N, M), where N >= 3 is the
          number of stimuli (i.e., the batch size), and M >= 2 is the
          number of unit activations per stimulus.

    Raises:
        ValueError: If the ``activations`` tensor does not meet the size
          and dimensionality requirements stated above.
    """
    
    if not isinstance(activations, torch.Tensor):
        raise ValueError(
            f"'activations' should be of type torch.Tensor, but is of type {type(activations)}.'"
        )
    
    if activations.dim() != 2:
        raise ValueError(
            f"'activations' should be 2-dimensional, but has {activations.dim()} dimensions."
        )

    if activations.size(dim=0) < 3:
        raise ValueError(
            "'activations' should contain activations for at least 3 stimuli."
        )

    if activations.size(dim=1) < 2:
        raise ValueError(
            "The number of unit activations per stimulus should be at least 2."
        )


def _is_vector(x: torch.Tensor) -> bool:
    """Check if the input is a ``torch.Tensor`` of dimension 1."""
    
    return isinstance(x, torch.Tensor) and x.dim() == 1


### Compute RDMs

* ``compute_rdm_euclidean`` &rarr; Compute an RDM using Euclidean distance.
* ``compute_rdm_correlation`` &rarr; Compute an RDM using (Pearson) correlation distance.
* ``compute_rdm`` &rarr; Compute an RDM using the specified method.


#### Euclidean Distance

We make use of the following identity in the ``compute_rdm_euclidean`` function: For two vectors $\mathbf{a}$ and $\mathbf{b}$,  
the squared Euclidean distance can be computed as follows:
$$
\lVert \mathbf{a} - \mathbf{b} \rVert_2^2 = \langle \mathbf{a} - \mathbf{b}, \mathbf{a} - \mathbf{b} \rangle = \lVert \mathbf{a} \rVert_2^2 + \lVert \mathbf{b} \rVert_2^2 - 2 \mathbf{a}^{\top} \mathbf{b} \,.
$$

If $\mathbf{a} = A_{i,:}^{\top}$ and $\mathbf{b} = A_{j,:}^{\top}$ correspond to two rows of an $N \times M$ matrix $A$ of activations, this identity becomes
$$
\lVert A_{i,:}^{\top} - A_{j,:}^{\top} \rVert_2^2 = \lVert A_{i,:}^{\top} \rVert_2^2 + \lVert A_{j,:}^{\top} \rVert_2^2 - 2 A_{i,:} A_{j,:}^{\top} = \lVert A_{i,:}^{\top} \rVert_2^2 + \lVert A_{j,:}^{\top} \rVert_2^2 - 2 (A A^{\top})_{i,j} \,.
$$

By defining a vector $\mathbf{d}$ like so
$$
\mathbf{d}_i = \sum_{m=1}^M A_{i,m}^2 = \lVert A_{i,:}^{\top} \rVert_2^2 \,, \quad i = 1, \ldots, N \,,
$$
the squared Euclidean distance between the activations elicited by the $i$-th and $j$-th stimulus, respectively, is given by the  
entry in position $(i, j)$ of the matrix
$$
\mathbf{d} + \mathbf{d}^{\top} - 2 A A^{\top},
$$
where the column- and row-vectors $\mathbf{d}$ and $\mathbf{d}^{\top}$, respectively, are broadcast to $N \times N$-matrices.

In [9]:
def compute_rdm_euclidean(
    activations: torch.Tensor,
    center_activations: bool = False,
    normalize_distances: bool = True,
    distance_type: Literal["squared", "non-squared"] = "squared"
) -> torch.Tensor:
    """Compute an RDM using Euclidean distance.

    Note:
        This function returns only the upper triangular matrix of the
        computed RDM in vectorized form (row-major order).

    Args:
        activations: The matrix of activations from which to compute the
          RDM.  Must be a 2-D tensor of size (N, M), where N >= 3 is the
          number of stimuli, and M >= 2 is the number of unit
          activations per stimulus.
        center_activations: Whether to center the activations for each
          stimulus before computing distances.
        normalize_distances: Whether to normalize the squared pairwise
          distances by the number M of unit activations per stimulus.
        distance_type: Whether to return squared or non-squared
          distances.

    Returns:
        The RDM (in vectorized form) computed from the data using
        Euclidean distance.
    """

    _validate_activations(activations)

    if distance_type not in ["squared", "non-squared"]:
        raise ValueError(
            f"'distance_type' should be either 'squared' or 'non-squared', but got {distance_type}."
        )

    if center_activations:
        activations -= activations.mean(dim=1, keepdim=True)
    
    # Compute squared pairwise distances
    norms_squared = torch.sum(torch.square(activations), dim=1, keepdim=True)
    distances_squared = norms_squared + norms_squared.T - 2 * torch.mm(activations, activations.T)

    # Clamp negative values (potential numerical inaccuracies) and extract RDM
    rdm = _get_upper_tri_matrix(torch.clamp(distances_squared, min=0.0))

    if normalize_distances:
        rdm = rdm / activations.size(dim=1)

    if distance_type == "non-squared":
        rdm = torch.sqrt(rdm)
    
    return rdm


#### Correlation Distance

In [10]:
def compute_rdm_correlation(
    activations: torch.Tensor,
    center_activations: bool = True
) -> torch.Tensor:
    """Compute an RDM using (Pearson) correlation distance.

    Note:
        This function returns only the upper triangular matrix of the
        computed RDM in vectorized form (row-major order).

    Args:
        activations: The matrix of activations from which to compute the
          RDM.  Must be a 2-D tensor of size (N, M), where N >= 3 is the
          number of stimuli, and M >= 2 is the number of unit
          activations per stimulus.
        center_activations: Whether to center the activations for each
          stimulus before computing distances.

    Returns:
        The RDM (in vectorized form) computed from the data using
        (Pearson) correlation distance.
    """

    _validate_activations(activations)

    if center_activations:
        activations -= activations.mean(dim=1, keepdim=True)

    rdm = 1 - _get_upper_tri_matrix(torch.corrcoef(activations))

    return rdm


#### Main Function

In [11]:
def compute_rdm(
    activations: torch.Tensor,
    method: str,
    **kwargs: Any
) -> torch.Tensor:
    """Compute an RDM using the specified method.

    Args:
        activations: The matrix of activations from which to compute the
        RDM.  Must be a 2-D tensor of size (N, M), where N >= 3 is the
          number of stimuli, and M >= 2 is the number of unit
          activations per stimulus.
        method: The method to use for computing the RDM.
        **kwargs: Additional keyword arguments to pass to the specific
          function used to compute the RDM.

    Returns:
        The RDM (in vectorized form) computed from the data using the
          specified method.
    """

    methods: Dict[str, Callable[..., torch.Tensor]] = {
        "euclidean": compute_rdm_euclidean,
        "correlation": compute_rdm_correlation,
    }

    if method not in methods:
        raise ValueError(
            f"'method' should be one of {list(methods.keys())}, but got {method}."
        )

    return methods[method](activations, **kwargs)


### Compare RDMs

* ``compare_rdm_cosine`` &rarr; Compare two RDMs using cosine similarity.
* ``compare_rdm`` &rarr; Compare two RDMs using the specified method.


#### Cosine Similarity

In [12]:
def compare_rdm_cosine(
    rdm1: torch.Tensor,
    rdm2: torch.Tensor
) -> torch.Tensor:
    """Compare two RDMs using cosine similarity.

    Args:
        rdm1: The first RDM in vectorized form.
        rdm2: The second RDM in vectorized form.

    Returns:
        The cosine similarity between the two RDMs.
    """

    if not (_is_vector(rdm1) and _is_vector(rdm2)):
        raise ValueError(
            "Both 'rdm1' and 'rdm2' should be tensors of dimension 1."
        )
    
    cosine_similarity = torch.dot(rdm1, rdm2) / (
        LA.vector_norm(rdm1, ord=2) * LA.vector_norm(rdm2, ord=2)
    )
    return cosine_similarity


#### Main Function

In [13]:
def compare_rdm(
    rdm1: torch.Tensor,
    rdm2: torch.Tensor,
    method: str,
    **kwargs: Any
) -> torch.Tensor:
    """Compare two RDMs using the specified method.

    Args:
        rdm1: The first RDM in vectorized form.
        rdm2: The second RDM in vectorized form.
        method: The method to use for comparing the RDMs.
        **kwargs: Additional keyword arguments to pass to the specific
          function used to compare the RDMs.

    Returns:
        The similarity between the two RDMs based on the specified
        method.
    """

    methods: dict[str, Callable[..., torch.Tensor]] = {
        "cosine": compare_rdm_cosine
    }

    if method not in methods:
        raise ValueError(
            f"'method' should be one of {list(methods.keys())}, but got {method}."
        )

    return methods[method](rdm1, rdm2, **kwargs)


### Custom Loss

* ``rsa_loss`` &rarr; A weighted loss combining cross-entropy and repr. similarity.

In [14]:
def rsa_loss(
    predictions: torch.Tensor,
    targets: torch.Tensor,
    activations1: torch.Tensor,
    activations2: torch.Tensor,
    method_compute: Callable[[torch.Tensor], torch.Tensor],
    method_compare: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
    weight_rsa_score: float,
    rsa_transform: Optional[Callable[[torch.Tensor], torch.Tensor]] = None
) -> Tuple[torch.Tensor, torch.Tensor]:
    """A weighted loss combining cross-entropy and repr. similarity.

    Note:
        Both, the ``method_compute`` and the ``method_compare``
        functions should take as arguments only the activations and
        RDMs, respectively.  Additional arguments must already be
        pre-bound to these functions using the ``partial`` function of
        the ``functools`` module.
    
    Args:
        predictions: The model predictions.
        targets: The target values.
        activations1: First set of activations.  Must be a 2-D tensor of
          size (N, M), where N >= 3 is the number of stimuli and M >= 2
          is the number of unit activations per stimulus.
        activations2: Second set of activations.  Must be a 2-D tensor
          of size (N, K), where N >= 3 is the number of stimuli and
          K >= 2 is the number of unit activations per stimulus.
        method_compute: The method used to compute the RDMs from the
          activations.
        method_compare: The method used to compare RDMs.
        weight_rsa_score: The weight attributed to the (transformed) RSA
          score in the weighted loss combination.
        rsa_transform: The transformation to apply to the raw RSA score.

    Returns:
        A tuple consisting of the raw RSA score between the two networks
        and the weighted loss.

    Raises:
        ValueError: If ``weight_rsa_score`` is outside the range [0, 1]
          or if one of the activation tensors does not meet the size and
          dimensionality requirements.
    """

    if not 0 <= weight_rsa_score <= 1:
        raise ValueError(
            "'weight_rsa_score' should be a float in the range [0, 1], "
            f"but got {weight_rsa_score}."
        )

    # Check activation tensors
    _validate_activations(activations1)
    _validate_activations(activations2)
    if activations1.size(dim=0) != activations2.size(dim=0):
        raise ValueError(
            "'activations1' and 'activations2' should have the same number of rows."
        )

    # Compute cross-entropy loss
    loss = F.cross_entropy(predictions, targets)

    # Compute RSA score
    rsa_score = method_compare(
        method_compute(activations1),
        method_compute(activations2)
    )

    # Transform RSA score & compute weighted loss
    rsa_score_transformed = rsa_score if rsa_transform is None else rsa_transform(rsa_score)
    weighted_loss = weight_rsa_score * rsa_score_transformed + (1 - weight_rsa_score) * loss
    
    return rsa_score, weighted_loss


## Experiments


### Penultimate Layer

In [15]:
cfg_dict_1 = cfg_dict.copy()

# Choose penultimate layer to use for computing RSA scores
cfg_dict_1["hooks"] = {
    "ref": "net.10",
    "train": "net.10"
}

# Set training parameters
cfg_dict_1["performance"]["patience"] = None
cfg_dict_1["training"]["num_epochs"] = 100

# Instantiate models
model_train = load_pretrained_lenet(is_training=True)
model_ref = load_pretrained_lenet(is_training=False)

# Set up optimizer
lr = 0.01
momentum = 0.8
weight_decay = 1e-3
optimizer = torch.optim.SGD(
    model_train.parameters(),
    lr=lr,
    momentum=momentum,
    weight_decay=weight_decay
)
optim_params = optimizer.state_dict()["param_groups"][0]
optim_params.pop("params")
cfg_dict_1["optimizer"]["params"] = optim_params

# Set up criterion
# NOTE: Cosine similarity returns values between -1 and 1, 0 indicating orthogonality of the two RDMs
method_compute = partial(
    compute_rdm,
    method="euclidean",
    center_activations = False,
    normalize_distances = True,
    distance_type = "squared"
)
method_compare = partial(
    compare_rdm,
    method="cosine"
)
rsa_transform = torch.abs
weight_rsa_score = 0.8
criterion = partial(
    rsa_loss,
    method_compute=method_compute,
    method_compare=method_compare,
    weight_rsa_score=weight_rsa_score,
    rsa_transform=rsa_transform
)

# Set paths
now = get_timestamp()
cfg_dict_1["paths"] = {
    "checkpoints": f"../out/lenet_fashionmnist/representational_similarity/test_run/{now}/checkpoints",
    "tensorboard": f"../out/lenet_fashionmnist/representational_similarity/test_run/{now}/tensorboard"
}
cfg_1 = OmegaConf.create(cfg_dict_1)

# Random seeds
set_seeds()

# Initialize trainer
representational_similarity_trainer = RepresentationalSimilarityTrainer(
    model_train=model_train,
    model_ref=model_ref,
    optimizer=optimizer,
    criterion=criterion,
    train_loader=TRAIN_LOADER,
    val_loader=VAL_LOADER,
    prediction_metrics=PREDICTION_METRICS,
    device=DEVICE,
    cfg=cfg_1
)

# Inspect training configuration & save config file
print(f"\033[1mConfiguration of Experiment\033[0m:\n\n{OmegaConf.to_yaml(cfg_1)}")
logs_dir = Path(f"../out/lenet_fashionmnist/representational_similarity/test_run/{now}/logs")
logs_dir.mkdir(parents=True, exist_ok=True)
OmegaConf.save(
    cfg_1,
    logs_dir / "config.yaml"
)

# Start training
representational_similarity_trainer.train()

# Remove all hooks
representational_similarity_trainer.remove_hooks()

Configuration of Experiment:

checkpoints:
  delete_previous: true
  save_best_model: true
  save_frequency: 5
dataloader:
  batch_size: 128
model:
  name: lenet
optimizer:
  name: SGD
  params:
    lr: 0.01
    momentum: 0.8
    dampening: 0
    weight_decay: 0.001
    nesterov: false
    maximize: false
    foreach: null
    differentiable: false
    fused: null
performance:
  dataset: Val
  higher_is_better: false
  keep_previous_best_score: true
  metric: Loss
  patience: null
tensorboard:
  updates_per_epoch:
    Train: 10
    Val: 10
training:
  resume_from: null
  num_epochs: 100
hooks:
  ref: net.10
  train: net.10
paths:
  checkpoints: ../out/lenet_fashionmnist/representational_similarity/test_run/2024-09_13-15-04-37/checkpoints
  tensorboard: ../out/lenet_fashionmnist/representational_similarity/test_run/2024-09_13-15-04-37/tensorboard

